# TensorFlow API Basics

Import TensorFlow API library, NumPy

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
print(tf.__version__)

2.4.0


- TF Constants
- TF Variables

In [4]:
x_np = np.array([1,2,3])
x_np

array([1, 2, 3])

In [5]:
# Constructing a constant tensor
c_tf = tf.constant(x_np, dtype=tf.int16)
c_tf

<tf.Tensor: shape=(3,), dtype=int16, numpy=array([1, 2, 3], dtype=int16)>

In [6]:
# Construct a variable tensor
x_tf = tf.Variable(np.random.randn(2,3), dtype=tf.float64)
x_tf

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[-0.56890229,  0.17860116, -1.69089287],
       [ 1.17218475,  0.40432219, -0.28149195]])>

We can convert both back to NumPy

In [7]:
c_tf.numpy()

array([1, 2, 3], dtype=int16)

In [8]:
x_tf.numpy()

array([[-0.56890229,  0.17860116, -1.69089287],
       [ 1.17218475,  0.40432219, -0.28149195]])

In [9]:
x_tf

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[-0.56890229,  0.17860116, -1.69089287],
       [ 1.17218475,  0.40432219, -0.28149195]])>

TF reimplements most of NumPy API.

Be careful:

1. Some functions are renamed.

2. TF API are really picky about data types.

In [13]:
np.repeat(np.reshape(x_np, (3,1)), 3, axis=1)

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [16]:
tf.repeat(tf.reshape(c_tf, (3,1)), 3, axis=1)

<tf.Tensor: shape=(3, 3), dtype=int16, numpy=
array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]], dtype=int16)>

In [19]:
# Perform NumPy addition between c_tf.numpy() + x_tf.numpy()
x_tf.numpy() + c_tf.numpy()

array([[0.43109771, 2.17860116, 1.30910713],
       [2.17218475, 2.40432219, 2.71850805]])

In [21]:
# Perform NumPy addition between c_tf.numpy() + x_tf.numpy()
try:
    x_tf + c_tf
except Exception as e:
    print(e)

cannot compute AddV2 as input #1(zero-based) was expected to be a double tensor but is a int16 tensor [Op:AddV2]


In [23]:
# Perform NumPy addition between c_tf.numpy() + x_tf.numpy()
try:
    print(x_tf + tf.cast(c_tf, tf.float64))
except Exception as e:
    print(e)

tf.Tensor(
[[0.43109771 2.17860116 1.30910713]
 [2.17218475 2.40432219 2.71850805]], shape=(2, 3), dtype=float64)


# Variables

TF Variables

```python
x = tf.Variable(...)

x.assign(val)     # assigns a value to x.
x.assign_add(val) # adds a value to x
x.assign_sub(val) # subtracts a value from x
```

The value object can be a tf.Tensor or np.NDArray.

In [29]:
print(x_tf)
x_tf.assign_add(np.array([[1,1,1],
                          [2,2,2]]))
print(x_tf)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[3.43109771, 4.17860116, 2.30910713],
       [9.17218475, 8.40432219, 7.71850805]])>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[ 4.43109771,  5.17860116,  3.30910713],
       [11.17218475, 10.40432219,  9.71850805]])>


# Auto-differentiation

Describe a computational graph as a function using different TF APIs.

In [33]:
def f(y):
    return tf.reduce_sum(y ** 2, axis=1)

In [36]:
print("x =")
print(x_tf)
print("f(x) = ")
print(f(x_tf))

x =
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[ 4.43109771,  5.17860116,  3.30910713],
       [11.17218475, 10.40432219,  9.71850805]])>
f(x) = 
tf.Tensor([ 57.4027269  327.51703108], shape=(2,), dtype=float64)


The computational graph is:

- $z = f(y)$
- $z[0] - z[1]$
- $\|z[0] - z[1]\|$

In [38]:
z = f(x_tf)
w = tf.abs(z[0] - z[1])

print("w =")
print(w)

w =
tf.Tensor(270.11430417850545, shape=(), dtype=float64)


1. Create an auto-diff context.


2. Perform all computations within the context.


3. The context can keep track of gradients for us.

---

The tape object `t` can compute the gradient of **any** scalar tensor with respect to **any** tensorflow **Variable** objects.

In [39]:
with tf.GradientTape() as t:
    y = tf.abs(f(x_tf)[0] - f(x_tf)[1])

In [40]:
# the tape context object t can compute the gradient
# of w w.r.t. x_tf

x_grad = t.gradient(y, x_tf)
x_grad

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[ -8.86219542, -10.35720232,  -6.61821425],
       [ 22.34436949,  20.80864438,  19.43701611]])>